In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
from IPython.display import clear_output
import matplotlib.pyplot as plt
from progressbar import progressbar as pb

data = pd.read_csv('/content/drive/My Drive/Embeddings/big_data_labels.csv', index_col=False)
data.head()

,0,1,2,3,4,5,label
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2,История
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2,История
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3,История
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1,История
4,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1,История


In [0]:
import pickle

with open('/content/drive/My Drive/Embeddings/elmo_embed_big.pickle', 'rb') as f:
    elmo_embs = pickle.load(f)

elmo_embs = np.array(elmo_embs)

with open('/content/drive/My Drive/Embeddings/bert_embed_big.pickle', 'rb') as f:
    bert_embs = pickle.load(f)

bert_embs = np.array(bert_embs)

with open('/content/drive/My Drive/Embeddings/fasttext_embed_big.pickle', 'rb') as f:
    fasttext_embs = pickle.load(f)

fasttext_embs = np.array(fasttext_embs)

with open('/content/drive/My Drive/Embeddings/rusvec_embed_big.pickle', 'rb') as f:
    rusvec_embs = pickle.load(f)

rusvec_embs = np.array(rusvec_embs)

In [0]:
def get_samples(data, embeddings, train_part=0.75):
    predata = np.zeros((embeddings.shape[0] * 4, embeddings.shape[2] * 2))
    target = []

    for i in range(len(data)):
        for j in range(1,5):
            cur_targ = 1 if j == int(data.iloc[i,5]) else 0
            cur_vec = np.hstack((embeddings[i,0], embeddings[i,j]))
            predata[i*4+j-1] = cur_vec
            target.append(cur_targ)
            
    all_inds = np.arange(embeddings.shape[0])
    train_size = int(len(all_inds) * train_part)
    train_inds = np.random.choice(all_inds, size=train_size, replace=False)
    test_inds = np.array(list(set(all_inds) - set(train_inds)))
    
    X_train, X_test, y_train, y_test = [],[],[],[]

    for i in range(len(train_inds)):
        for j in range(4):
            X_train.append(predata[train_inds[i] * 4 + j])
            y_train.append(target[train_inds[i] * 4 + j])

    for i in range(len(test_inds)):
        for j in range(4):
            X_test.append(predata[test_inds[i] * 4 + j])
            y_test.append(target[test_inds[i] * 4 + j])
            
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [0]:
def get_batch(X_val, y_val, batch_sz=16):
    X_batches, y_batches = [], []
    
    samples_num = len(X_val) // 4
    samples_inds = np.arange(samples_num)
    np.random.shuffle(samples_inds)
    
    cur_x, cur_y = [], []
    
    for i, cur_ind in enumerate(samples_inds):
        if i % batch_sz < batch_sz-1:
            cur_x.extend(X_val[cur_ind:cur_ind+4])
            cur_y.extend(y_val[cur_ind:cur_ind+4])
        else:
            cur_x.extend(X_val[cur_ind:cur_ind+4])
            cur_y.extend(y_val[cur_ind:cur_ind+4])
            X_batches.append(cur_x)
            y_batches.append(cur_y)
            cur_x, cur_y = [], []
            
    X_batches.append(cur_x)
    y_batches.append(cur_y)
    
    return np.array(X_batches), np.array(y_batches)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

def get_score(preds, targets):
    preds, targets = np.concatenate(preds, axis=0), np.concatenate(targets, axis=0)
    y_pred_splitted, y_target_splitted = [], []
    
    for i in range(len(preds) // 4):
        y_pred_splitted.append(np.argmax(preds[i*4:i*4+4]))
        y_target_splitted.append(np.argmax(targets[i*4:i*4+4]))
        
        
    accuracy = accuracy_score(y_target_splitted, y_pred_splitted)
    precision_macro = precision_score(y_target_splitted, y_pred_splitted, average='macro')
    precision_micro = precision_score(y_target_splitted, y_pred_splitted, average='micro')
    f1_macro = f1_score(y_target_splitted, y_pred_splitted, average='macro')
    f1_micro = f1_score(y_target_splitted, y_pred_splitted, average='micro')
    
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [0]:
import torch
import torch.nn as nn

class SiameseNet(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        self.left = nn.Sequential(
                                nn.Linear(embed_size, embed_size * 3 // 4),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(embed_size * 3 // 4, embed_size // 2),
                                nn.ReLU(),
                                nn.Dropout(0.3))
            
        self.right = nn.Sequential(
                                nn.Linear(embed_size, embed_size * 3 // 4),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(embed_size * 3 // 4, embed_size // 2),
                                nn.ReLU(),
                                nn.Dropout(0.3))
        
        self.center = nn.Sequential(
                                nn.Linear(embed_size, embed_size // 2),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(embed_size // 2, embed_size // 4),
                                nn.ReLU(),
                                nn.Dropout(0.3),
                                nn.Linear(embed_size // 4, 1),
                                nn.Sigmoid())
            
    def forward(self, q, a):
        left = self.left(q)
        right = self.right(a)
        x = torch.cat((left, right), 1)
        out = self.center(x)
        
        return out

In [0]:
def train(model, optimizer, X_train, y_train, batch_size):
    train_loss, all_preds, all_tests = [], [], []
    model.train()
    
    X_batches, y_batches = get_batch(X_train, y_train, batch_size)

    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        embed_shape_tr = X_batch.shape[1] // 2
        
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
    
        optimizer.zero_grad()
        y_pred = model(X_batch[:,:embed_shape_tr], X_batch[:, embed_shape_tr:])[:,0]

        loss = loss_fn(y_pred, y_batch.type(torch.cuda.FloatTensor))
        
        train_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        all_preds.append(y_pred.detach().cpu().numpy())
        all_tests.append(y_batch.cpu().numpy())
    
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(train_loss), list(score)

In [0]:
def validate(model, optimizer, X_test, y_test, batch_size):
    val_loss, all_preds, all_tests = [], [], []
    model.eval()
    
    X_batches, y_batches = get_batch(X_test, y_test, batch_size)
  
    
    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        embed_shape_tr = X_batch.shape[1] // 2
        
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
    
        y_pred = model(X_batch[:,:embed_shape_tr], X_batch[:, embed_shape_tr:])[:,0]
        
        loss = loss_fn(y_pred, y_batch.type(torch.cuda.FloatTensor))
        
        val_loss.append(loss.item())
        all_preds.append(y_pred.detach().cpu().numpy())
        all_tests.append(y_batch.cpu().numpy())
        
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(val_loss), list(score)

In [0]:
def launch_net(model, optimizer, X_train, y_train, X_test, y_test,
               train_batch=64, test_batch=16, epochs=25, graph=False):
    all_train_loss, all_val_loss = [], []
    all_train_acc, all_val_acc = [], []
    all_scores = []
    max_acc = 0

    for epoch in pb(range(epochs)):
        cur_train = train(model, optimizer, X_train, y_train, train_batch)
        cur_val = validate(model, optimizer, X_test, y_test, test_batch)
        all_train_loss.append(cur_train[0])
        all_train_acc.append(cur_train[1][0])
        all_val_loss.append(cur_val[0])
        all_val_acc.append(cur_val[1][0])
        all_scores.append(cur_val[1])
        max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
        clear_output()
        print('Epoch {}:'.format(epoch+1))
        print("Train loss: {}".format(cur_train[0]))
        print("Validation loss: {}".format(cur_val[0]))
        print("Train accuracy: {}".format(cur_train[1][0]))
        print("Validation accuracy: {}".format(cur_val[1][0]))
        print("Max validation accuracy: {}".format(max_acc))
        
        if graph:
          f, axarr = plt.subplots(1,2, figsize=(20,10))

          axarr[0].plot(all_train_loss, label='Train loss')
          axarr[0].plot(all_val_loss, label='Val loss')
          axarr[0].grid()
          axarr[0].legend(fontsize=15)
          axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

          axarr[1].plot(all_train_acc, label='Train accuracy')
          axarr[1].plot(all_val_acc, label='Val accuracy')
          axarr[1].grid()
          axarr[1].legend(fontsize=15)
          axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
          plt.show()
        
    all_scores = np.max(np.array(all_scores), axis=0)
    return all_scores

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
epochs = 20

## Fasttext

In [0]:
embeddings = fasttext_embs
X_train, X_test, y_train, y_test = get_samples(data, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [14]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:20 Time:  0:00:20


Epoch 20:
Train loss: 0.14117038903104795
Validation loss: 1.9016134706939138
Train accuracy: 0.7611179110567116
Validation accuracy: 0.33190578158458245
Max validation accuracy: 0.3435301315386968


## ELMO

In [0]:
embeddings = elmo_embs
X_train, X_test, y_train, y_test = get_samples(data, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [17]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:38 Time:  0:00:38


Epoch 20:
Train loss: 0.08006460126489401
Validation loss: 2.205169639354799
Train accuracy: 0.7833537331701347
Validation accuracy: 0.330070357907617
Max validation accuracy: 0.3410828999694096


## BERT

In [0]:
embeddings = bert_embs
X_train, X_test, y_train, y_test = get_samples(data, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [30]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:27 Time:  0:00:27


Epoch 20:
Train loss: 0.17287928373976189
Validation loss: 1.510831158335616
Train accuracy: 0.7447980416156671
Validation accuracy: 0.3049862343224228
Max validation accuracy: 0.31263383297644537


## RusVectores

In [0]:
embeddings = rusvec_embs
X_train, X_test, y_train, y_test = get_samples(data, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [23]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 32, epochs)
results_dict['RusVectores'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:24 Time:  0:00:24


Epoch 20:
Train loss: 0.20294037054885516
Validation loss: 1.8476384843437417
Train accuracy: 0.6971644226846185
Validation accuracy: 0.28785561333741205
Max validation accuracy: 0.30926888956867543


In [31]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.309269,0.343530,0.341083,0.312634
Precision Macro,0.301699,0.340899,0.341841,0.312338
Precision Micro,0.309269,0.343530,0.341083,0.312634
F1 Macro,0.298062,0.336091,0.335623,0.305735
F1 Micro,0.309269,0.343530,0.341083,0.312634


# История

In [32]:
hist_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'История':
    hist_inds.append(i)
len(hist_inds)

2498

In [0]:
elmo_embs_hist = elmo_embs[hist_inds]
bert_embs_hist = bert_embs[hist_inds]
fasttext_embs_hist = fasttext_embs[hist_inds]
rusvec_embs_hist = rusvec_embs[hist_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'История']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_hist
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [37]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:04 Time:  0:00:04

Epoch 20:
Train loss: 0.09318248865505059
Validation loss: 2.2431993067264555
Train accuracy: 0.791777896422851
Validation accuracy: 0.2784
Max validation accuracy: 0.2976


## ELMO

In [0]:
embeddings = elmo_embs_hist
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [40]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Epoch 20:
Train loss: 0.04336594417691231
Validation loss: 2.816913029551506
Train accuracy: 0.799252536038441
Validation accuracy: 0.2624
Max validation accuracy: 0.288


## BERT

In [0]:
embeddings = bert_embs_hist
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [50]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:05 Time:  0:00:05


Epoch 20:
Train loss: 0.08174781364699205
Validation loss: 2.3223167326301337
Train accuracy: 0.800320341697811
Validation accuracy: 0.2432
Max validation accuracy: 0.2688


## RusVectores

In [0]:
embeddings = rusvec_embs_hist
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [53]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 32, epochs)
results_dict['RusVectores'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


Epoch 20:
Train loss: 0.15397281100352606
Validation loss: 1.916767317056656
Train accuracy: 0.7052856380138814
Validation accuracy: 0.2272
Max validation accuracy: 0.28


In [54]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.280000,0.297600,0.288000,0.268800
Precision Macro,0.262019,0.296237,0.285784,0.268735
Precision Micro,0.280000,0.297600,0.288000,0.268800
F1 Macro,0.259883,0.288128,0.278913,0.260601
F1 Micro,0.280000,0.297600,0.288000,0.268800


# Медицина

In [55]:
med_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'Медицина':
    med_inds.append(i)
len(med_inds)

4013

In [0]:
elmo_embs_med = elmo_embs[med_inds]
bert_embs_med = bert_embs[med_inds]
fasttext_embs_med = fasttext_embs[med_inds]
rusvec_embs_med = rusvec_embs[med_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'Медицина']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_med
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [60]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Epoch 20:
Train loss: 0.10185004056741793
Validation loss: 2.03264029631539
Train accuracy: 0.7813226985709538
Validation accuracy: 0.3137450199203187
Max validation accuracy: 0.34163346613545814


## ELMO

In [0]:
embeddings = elmo_embs_med
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [63]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:09 Time:  0:00:09


Epoch 20:
Train loss: 0.089671300680493
Validation loss: 2.5734705868221464
Train accuracy: 0.7919574609504819
Validation accuracy: 0.3247011952191235
Max validation accuracy: 0.34163346613545814


## BERT

In [0]:
embeddings = bert_embs_med
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [66]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:08 Time:  0:00:08


Epoch 20:
Train loss: 0.21424539519163469
Validation loss: 1.6058398485183716
Train accuracy: 0.7494184114323695
Validation accuracy: 0.2908366533864542
Max validation accuracy: 0.34860557768924305


## RusVectores

In [0]:
embeddings = rusvec_embs_med
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [69]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 32, epochs)
results_dict['RusVectores'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Epoch 20:
Train loss: 0.1884144519766172
Validation loss: 2.126295391470194
Train accuracy: 0.7291458956463942
Validation accuracy: 0.27191235059760954
Max validation accuracy: 0.2848605577689243


In [70]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.284861,0.341633,0.341633,0.348606
Precision Macro,0.270730,0.338213,0.338907,0.352563
Precision Micro,0.284861,0.341633,0.341633,0.348606
F1 Macro,0.268761,0.332769,0.334228,0.344391
F1 Micro,0.284861,0.341633,0.341633,0.348606


# Биология

In [71]:
bio_inds = []

for i in range(len(data)):
  if data.iloc[i,6] == 'Биология':
    bio_inds.append(i)
len(bio_inds)

2184

In [0]:
elmo_embs_bio = elmo_embs[bio_inds]
bert_embs_bio = bert_embs[bio_inds]
fasttext_embs_bio = fasttext_embs[bio_inds]
rusvec_embs_bio = rusvec_embs[bio_inds]

In [0]:
results_dict = {'RusVectores': [], 'FastText': [], 'ELMO': [], 'BERT': []}
data2 = data[data['label'] == 'Биология']
epochs = 20

## FastText

In [0]:
embeddings = fasttext_embs_bio
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [76]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['FastText'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


Epoch 20:
Train loss: 0.09104758902237965
Validation loss: 2.628712337357657
Train accuracy: 0.7771672771672772
Validation accuracy: 0.24358974358974358
Max validation accuracy: 0.2619047619047619


## ELMO

In [0]:
embeddings = elmo_embs_bio
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [79]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['ELMO'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:05 Time:  0:00:05


Epoch 20:
Train loss: 0.054728186617677026
Validation loss: 3.3171062537602016
Train accuracy: 0.8125763125763126
Validation accuracy: 0.22527472527472528
Max validation accuracy: 0.24175824175824176


## BERT

In [0]:
embeddings = bert_embs_bio
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [82]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 16, epochs)
results_dict['BERT'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


Epoch 20:
Train loss: 0.08146627619862556
Validation loss: 2.2370952538081577
Train accuracy: 0.7722832722832723
Validation accuracy: 0.2838827838827839
Max validation accuracy: 0.3131868131868132


## RusVectores

In [0]:
embeddings = rusvec_embs_bio
X_train, X_test, y_train, y_test = get_samples(data2, embeddings)

In [0]:
model = SiameseNet(embeddings.shape[2]).cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCELoss()

In [85]:
results = launch_net(model, optimizer, X_train, y_train, X_test, y_test, 64, 32, epochs)
results_dict['RusVectores'] = results

100% (20 of 20) |########################| Elapsed Time: 0:00:03 Time:  0:00:03


Epoch 20:
Train loss: 0.16261749084179217
Validation loss: 2.010254999001821
Train accuracy: 0.7167277167277167
Validation accuracy: 0.29304029304029305
Max validation accuracy: 0.3058608058608059


In [86]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.305861,0.261905,0.241758,0.313187
Precision Macro,0.288833,0.258148,0.242055,0.312145
Precision Micro,0.305861,0.261905,0.241758,0.313187
F1 Macro,0.284157,0.250001,0.233148,0.304496
F1 Micro,0.305861,0.261905,0.241758,0.313187
